In [1]:
#r "nuget: Microsoft.Extensions.AI, 9.9.0"

Installed Packages Microsoft.Extensions.AI, 9.9.0

In [2]:
#r "nuget: Azure.AI.Agents.Persistent, 1.2.0-beta.5"
#r "nuget: Azure.Identity, 1.15.0"
#r "nuget: System.Linq.Async, 6.0.3"

Installed Packages Azure.AI.Agents.Persistent, 1.2.0-beta.5 Azure.Identity, 1.15.0 System.Linq.Async, 6.0.3

In [4]:
#r "C:\Users\kinfeylo\Documents\Agent\agent-framework\dotnet\src\Microsoft.Agents.AI.AzureAI\bin\Debug\net9.0\Microsoft.Agents.AI.AzureAI.dll"

In [5]:
#r "C:\Users\kinfeylo\Documents\Agent\agent-framework\dotnet\src\Microsoft.Agents.AI\bin\Debug\net9.0\Microsoft.Agents.AI.dll"

In [6]:
#r "nuget: Microsoft.Extensions.AI.OpenAI, 9.9.0-preview.1.25458.4"

Installed Packages Microsoft.Extensions.AI.OpenAI, 9.9.0-preview.1.25458.4

In [7]:
#r "nuget: DotNetEnv, 3.1.1"

Installed Packages DotNetEnv, 3.1.1

In [8]:
using System;
using System.Linq;
using Azure.AI.Agents.Persistent;
using Azure.Identity;
using Microsoft.Agents.AI;

In [9]:
 using DotNetEnv;

In [10]:
Env.Load("../../../../.env");

In [11]:
var azure_foundry_endpoint = Environment.GetEnvironmentVariable("AZURE_AI_PROJECT_ENDPOINT") ?? throw new InvalidOperationException("AZURE_AI_PROJECT_ENDPOINT is not set.");
var azure_foundry_model_id = Environment.GetEnvironmentVariable("AZURE_AI_MODEL_DEPLOYMENT_NAME") ?? "gpt-4.1-mini";

In [12]:
string pdfPath = "../../files/demo.md";

In [13]:
using System.IO;

async Task<Stream> OpenImageStreamAsync(string path)
{
	return await Task.Run(() => File.OpenRead(path));
}

var pdfStream = await OpenImageStreamAsync(pdfPath);

In [14]:
var persistentAgentsClient = new PersistentAgentsClient(azure_foundry_endpoint, new AzureCliCredential());

In [15]:
PersistentAgentFileInfo fileInfo = await persistentAgentsClient.Files.UploadFileAsync(pdfStream, PersistentAgentFilePurpose.Agents, "demo.md");

In [16]:
PersistentAgentsVectorStore fileStore =
            await persistentAgentsClient.VectorStores.CreateVectorStoreAsync(
                [fileInfo.Id],
                metadata: new Dictionary<string, string>() { { "agentkey", bool.TrueString } });

In [17]:
PersistentAgent agentModel = await persistentAgentsClient.Administration.CreateAgentAsync(
            azure_foundry_model_id,
            name: "GraphRAGAgent",
            tools: [new FileSearchToolDefinition()],
            instructions: "You are an AI assistant that helps people find information in a set of documents. Use the File Search tool to look up relevant information from the files when needed to answer user questions. If you don't know the answer, just say you don't know. Do not make up answers.",
            toolResources: new()
            {
                FileSearch = new()
                {
                    VectorStoreIds = { fileStore.Id },
                }
            },
            metadata: new Dictionary<string, string>() { { "agentkey", bool.TrueString } });

In [18]:
AIAgent agent = await persistentAgentsClient.GetAIAgentAsync(agentModel.Id);

In [19]:
AgentThread thread = agent.GetNewThread();

In [20]:
Console.WriteLine(await agent.RunAsync("What's graphrag", thread));

GraphRAG is an AI-based content interpretation and search capability that uses large language models (LLMs) to parse data, create a knowledge graph, and answer user questions about a private dataset provided by the user. It is designed to connect information across large volumes of data and use these connections to answer complex questions that might be difficult or impossible to answer with traditional keyword or vector-based search methods. GraphRAG can also provide insights on thematic questions such as identifying top themes in a dataset.

Its intended use includes supporting critical information discovery and analysis where information spans many documents, and the data might be noisy or mixed with misinformation. It is aimed at users trained in responsible analytic approaches and critical reasoning, enabling high degrees of insight on complex topics with human expert verification of the generated responses.

GraphRAG operates by constructing an index of the input data, which is a